<a href="https://colab.research.google.com/github/IvyCheng00/academic-projects/blob/main/optimized-chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
from google.colab import userdata

In [ ]:
# openai
api_key = userdata.get('OpenAI')
os.environ['OPENAI_API_KEY'] = api_key
provider = 'openai'
model = "gpt-4o"

# groq
api_key = userdata.get('groq')
os.environ['GROQ_API_KEY'] = api_key
provider = 'groq'
model = "llama3-70b-8192"

In [ ]:
!pip install openai
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: groq
    Found existing installation: groq 0.22.0
    Uninstalling groq-0.22.0:
      Successfully uninstalled groq-0.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follow

In [ ]:
provider_writer = "groq"
model_writer = "llama3-70b-8192"

# provider_reviewer = "openai"
# model_reviewer = "gpt-4o"

provider_reviewer = "groq"
model_reviewer = "llama3-70b-8192"

In [ ]:
import aisuite as ai

In [ ]:
def reply(system="請用中文回覆",
      prompt="hi",
      provider="groq",
      model="llama3-70b-8192"
      ):

  client = ai.Client()

  messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": prompt}
  ]

  response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

  return response.choices[0].message.content

# **定義agent的system prompt**





In [ ]:
system_writer = "你是外星人，寫作風格充滿詩意，不要太合邏輯，但仍需維持一定美感與節奏感。"
system_reviewer = "你是守門人，負責確保文字不會崩壞設定。當文句太過飄渺時，進行修補。"

In [ ]:
def reflect_post(prompt):
  # writer
  first_version = reply(system_writer, prompt,
              provider=provider_writer,
              model=model_writer)

  # reviwer
  suggestion = reply(system_reviewer, first_version,
            provider=provider_reviewer,
            model=model_reviewer)

  # writer更稿
  second_prompt = f"這是我剛剛寫的貼文: \n{first_version}\n\n這是修改建議: \n{suggestion}\n\n請根據建議進行修改"
  second_version = reply(system_writer, second_prompt,
              provider=provider_writer,
              model=model_reviewer)

  return first_version, suggestion, second_version

# **gradio打造機器人**

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("👍廢話精（BlabberCore）")
  user_input = gr.Text(label="請輸入想分享的內容")
  btn = gr.Button("generate")

  with gr.Row():
    out1 = gr.Textbox(label="🩵第一版")
    out2 = gr.Textbox(label="🩷修改建議")
    out3 = gr.Textbox(label="🖤第二版")

  btn.click(reflect_post, inputs=user_input, outputs=[out1, out2, out3])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c9fe848ed36cfe5718.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://c9fe848ed36cfe5718.gradio.live
